# Eksperiment: Genre-rum

Vi har en idé om hvad *afstanden* er mellem to punkter.
Hvis vi har to prikker på et stykke papir, véd vi hvordan måler afstanden mellem dem.
Hvis vi har to steder i vores dagligstue, har vi også en idé om hvad deres afstand er.

Hvordan kan vi tale om afstand mellem to bøger? Vi vil nok forvente, at bøger indenfor samme genre ligger tæt på hinanden, eksempler:

- Afstanden mellem "Peter Plys" og "Frøken Smillas fornemmelse for sne" er nok større end afstanden mellem "Cirkeline" og "Cykelmyggen Egon".
- En rejsebog om Berlin er nok tættere på en rejsebog om Paris, end på en håndarbejdsbog.
- Bøger af samme forfatter ligger nok i nærheden af hinanden.

Forestil dig at vi har et genre-rum, hvor der er et punkt for hver eneste bog, og vi kan måle afstanden mellem dem. Dette vil give nye muligheder for at gå på opdagelse i litteraturen. Anbefalinger dukker op ved, at finde de nærmeste nabopunkter. Genre består af punkter i nærheden af hinanden. Dette gør at vi kan bruge computeren til at udforske litteraturen.

På papir, i vores dagligstue, såvel som i genrerummet, kan afstand defineres matematisk således: Hvis vi har to punkter, $a$ og $b$, så er afstanden mellem dem $\sqrt{(a-b)^2}$.
I én dimension er det afstanden mellem to tal. Eksempel: afstanden mellem $1$ og $3$ er $2$, hvilket kan udregnes som $\sqrt{(1 - 3)^2} = \sqrt{(-2)^2} = \sqrt{4} = 2$.
I to dimensioner er det afstand mellem to prikker på et stykke papir, også kaldet den "Euklidiske afstand", $\sqrt{(x_a - x_b)^2 + (y_a - y_b)^2}$. Eksempel: afstanden mellem koordinaterne $(1,2)$ og $(4,6)$ er $5$ hvilket vi kan udregne som $\sqrt{(1-4)^2 + (2-6)^2} = \sqrt{3^2 + 4^2} = \sqrt{9 + 16} = \sqrt{25} = 5$.
I tre dimensioner er det afstanden mellem to punkter i rummet, i.e. $\sqrt{(x_a - x_b)^2 + (y_a - y_b)^2 + (z_a - z_b)^2}$, - og det fortsætter på samme måde i fire, fem, seks, ... dimensioner. 

I matematik, kalder vi ofte koordinaterne for *vektorer*. Eksempelvis er en 5-dimensionel vektor, blot en liste af fem tal, og en 100-dimensionel vektor er en liste af hundrede tal.

Hvordan skaber vi et sådan genrerum? Vi har en masse statistik om lån på bibliotekerne. Hvis vi antager at man ofte låner indenfor samme genre, så kan computeren ud fra disse data udregne et genrerum.

Jeg har udregnet et sådan genrerum for 10.000 biblioteksmaterialer, og i det følgende vil vi undersøge, om afstanden mellem bøger i genrerummet giver mening.

## Eksperimenter med afstand mellem bøger

Når vi laver et program, må vi først fortælle computeren, hvilken funktionalitet vi har brug for: `bibdata` indeholder bibliografiske data, og genrerummet, som jeg har beregnet. `numpy` indeholder matematikfunktionalitet:

In [1]:
import bibdata
import numpy

Eksempelvis er der en funktion, som returnere titel/forfatter, hvis vi kommer med nummeret på et biblioteksmateriale.

In [2]:
bibdata.title_creator(8955)

'Peter Plys : komplet samling fortællinger og digte - A. A. Milne'

Herover ser vi at bog nummer $8955$ er "Peter Plys". Ligeledes kan vi se de øvrige bøger, som vi vil eksperimentere med herunder:

In [3]:
[(book_number, bibdata.title_creator(book_number))
   for book_number in [8955, 8214, 616, 580, 149, 278, 126, 29, 688]]

[(8955, 'Peter Plys : komplet samling fortællinger og digte - A. A. Milne'),
 (8214, 'Frøken Smillas fornemmelse for sne : roman - Peter Høeg'),
 (616, 'Cirkeline bliver til - Hanne Hastrup'),
 (580, 'Cykelmyggen Egon - Flemming Quist Møller'),
 (149, 'Turen går til Berlin - Therkelsen Kirstine'),
 (278, 'Turen går til Paris - Aske Munck'),
 (126, 'Alt om håndarbejdes strikkemagasin - '),
 (29, '1Q84 - Haruki Murakami'),
 (688, 'Kafka på stranden - Haruki Murakami')]

Vi kan finde punktet i genrerummet for en bog via `bibdata.genres`. Selve genrerummet er 100-dimensionelt, så vektoren består af 100 tal. I programmering kalde man ofte vektorer for *arrays*.

In [4]:
bibdata.genres[8955]

array([-0.00555056,  0.02921515,  0.02892058, -0.00930832,  0.0100181 ,
        0.02586022,  0.07639047,  0.0186474 , -0.00086742,  0.06219033,
        0.00679732,  0.11657441,  0.10813324, -0.02801489,  0.09085855,
        0.17314938,  0.01396204,  0.00813998,  0.02280807, -0.070943  ,
        0.10048762,  0.19593475,  0.07772335, -0.02303249, -0.02877597,
        0.0482331 ,  0.2106633 ,  0.10839563, -0.06060356,  0.04430847,
        0.00402697,  0.08289498,  0.13924087, -0.27573585,  0.2362854 ,
       -0.06875856,  0.16701048, -0.12419402, -0.092553  , -0.2432362 ,
       -0.04190033,  0.0515306 , -0.13153618,  0.02286854,  0.17851359,
        0.04017429, -0.11496432, -0.19422885, -0.29368186, -0.23722212,
       -0.05779668,  0.11127474, -0.00292388,  0.11682518, -0.04726336,
        0.24068264,  0.11335513, -0.00626563, -0.06929475, -0.06640247,
       -0.02567991, -0.0427829 ,  0.04892466, -0.00911773, -0.04064843,
        0.04555216,  0.02176529,  0.01931233,  0.05836495, -0.05

Som det næste vil vi definere en funktion, `distance`, som udregner afstanden mellem to punkter/vektorer. Denne bruger funktionen `numpy.linalg.norm(v)`, der udregner $\sqrt{v^2}$. Navnet `linalg` står for "linær algebra", som er den del af matematikken der blandt andet handler om at regne med vektorer. 

In [5]:
def distance(a, b):
    return numpy.linalg.norm(a - b)

Vi afprøver derefter funktionen ved, at finde afstanden mellem $(1, 2)$ og $(4, 6)$. Dette skal være $5$, ligesom vi udregnede tidligere. Her bruger vi `numpy.array`, som laver en liste af tal om til en vektor, så computeren kan regne på den.

In [6]:
distance(numpy.array([1, 2]), numpy.array([4, 6]))

5.0

Afstanden mellem "Peter Plys" og "Frøken Smilla" er:

In [7]:
distance(bibdata.genres[8955], bibdata.genres[8214])

1.3247944045025537

Afstanden mellem "Cirkeline" og "Cykelmyggen Egon" er:

In [8]:
distance(bibdata.genres[616], bibdata.genres[580])

1.0181284635824785

Afstanden mellem rejsebøger om "Berlin" og "Paris" er:

In [9]:
distance(bibdata.genres[149], bibdata.genres[278])

0.25173324294787786

Afstanden mellem rejsebog om "Berlin" og en håndarbejdsbog er:

In [10]:
distance(bibdata.genres[149], bibdata.genres[126])

1.3969369158183382

Afstanden mellem to bøger af "Haruki Murakami" er:

In [11]:
distance(bibdata.genres[29], bibdata.genres[688])

0.4369157294393759

Konklusionen er: at afstanden mellem bøger i genrerummet faktisk giver mening. De tre forventninger jeg formulerede i starten af kapitlet holdt. 

Vi kan observere, at rejsebøgerne ligger tættere på hinanden end Murakami-bøgerne, og at afstanden mellem Cirkeline og Cykelmyggen Egon er relativt stor. Dette ville jeg ikke have forudset, men den slags udsving er forventelige.

Bemærk også, at forventningerne blev formuleret før eksperimenterne blev programmeret og kørt. Når man laver data science / videnskabelige eksperimenter, gælder det om først at formulere hypotese, og hvorledes man kan teste den, - og derefter, at udføre testen for, at se om hypotesen faktisk holder.

## Litterærer anbefalinger

Lad os vælge en bog, og derefter finde afstanden fra denne til alle andre bøger. Når vi skriver `[0:10]` betyder det at vi kun viser de første 10 resultater, i stedet for hele listen.

In [12]:
choosen_book = 8955
print(bibdata.title_creator(choosen_book))

distances_from_peter_plys = [
    (distance(bibdata.genres[choosen_book], bibdata.genres[other_book]), 
        bibdata.title_creator(other_book)) 
    for other_book in range(0, 10000)
]
distances_from_peter_plys[0:10]

Peter Plys : komplet samling fortællinger og digte - A. A. Milne


[(1.4011684494447065, 'Fifty shades - E. L. James'),
 (1.4332511957653991, 'Journal 64 : krimithriller - Jussi Adler-Olsen'),
 (1.4319693083341756, 'Marco effekten : krimithriller - Jussi Adler-Olsen'),
 (1.4377953467714748, 'Taynikma - Jan Kjær (f. 1971)'),
 (1.4561631856697466, 'De glemte piger : krimi - Sara Blædel'),
 (1.4390826249649267, 'Den grænseløse : krimithriller - Jussi Adler-Olsen'),
 (1.2877310264792368, 'Vildheks - Lene Kꜳberbøl'),
 (1.4363966653595654, 'Dødesporet : krimi - Sara Blædel'),
 (1.4501109638315552, 'Dødsenglen : krimi - Sara Blædel'),
 (1.4287816916279745, 'Flaskepost fra P : krimithriller - Jussi Adler-Olsen')]

Hvis vi nu sorterer listen af bøger afstanden til den valgte bog, får vi en liste af anbefalinger.

In [13]:
sorted(distances_from_peter_plys)[0:10]

[(0.0, 'Peter Plys : komplet samling fortællinger og digte - A. A. Milne'),
 (0.53454719612995338,
  'Bogen om Emil fra Lønneberg : samlet udgave med alle historierne om Emil fra Lønneberg - Astrid Lindgren'),
 (0.53718166508329679,
  'Astrid Lindgrens allerbedste historier - Ingrid Vang Nyman'),
 (0.54053996866025411, 'Mumitrolden : de samlede striber - Tove Jansson'),
 (0.5654914190345337,
  'Klatremus og de andre dyr i Hakkebakkeskoven - Thorbjørn Egner'),
 (0.57263271765819757, 'Pippi Langstrømpe går om bord - Astrid Lindgren'),
 (0.57288576301783933, 'Bogen om Pippi Langstrømpe - Astrid Lindgren'),
 (0.58822503022521333,
  'Anne Marie Helger læser Vinden i piletræerne - Anne Marie Helger'),
 (0.60406738110296454, 'Pippi Langstrømpe i Sydhavet - Astrid Lindgren'),
 (0.61582753873808027,
  'Han er her endnu - Emil fra Lønneberg - Astrid Lindgren')]

Vi kan nu definerer dette som en funktion, som vi kan bruge senere. Denne funktion kan så bruges til at udforske anbefalinger til forskellige bøger:

In [14]:
def recommendations(choosen_book):
    return [recommendation[1] for recommendation in
        sorted([
            (distance(bibdata.genres[choosen_book], bibdata.genres[other_book]), 
                bibdata.title_creator(other_book)) 
            for other_book in range(0, 10000)])]

In [15]:
recommendations(8214)[0:20]

['Frøken Smillas fornemmelse for sne : roman - Peter Høeg',
 'Den kroniske uskyld - Klaus Rifbjerg',
 'Vinter-Eventyr - Karen Blixen',
 'Kongens Fald - Johannes V. Jensen (f. 1873)',
 'Ved Vejen - Herman Bang',
 'Rend mig i traditionerne - Leif Panduro',
 'Kronprinsessen : roman - Hanne-Vibeke Holst',
 'Det forsømte forår - Hans Scherfig',
 'Drageløberen - Khaled Hosseini',
 'Pelle Erobreren : barndom - Martin Andersen Nexø',
 'Løgneren - Martin A. Hansen, f. 1909',
 'Livlægens besøg : roman - Per Olov Enquist',
 'Kunsten at græde i kor : roman - Erling Jepsen (f. 1956)',
 'Den som blinker er bange for døden : roman - Knud Romer',
 'Åndernes hus - Chili Turèll',
 'Den gamle mand og havet - Ernest Hemingway',
 'De Elendige - Victor Hugo',
 'Fiskerne - Hans Kirk (f. 1898)',
 'Syv fantastiske fortællinger - Karen Blixen',
 'Thereses tilstand : roman - Hanne-Vibeke Holst']

In [16]:
recommendations(616)[0:20]

['Cirkeline bliver til - Hanne Hastrup',
 'Godmorgen Cirkeline - Hanne Hastrup',
 'Cirkeline - tæl til 10 - Hanne Hastrup',
 'Godnat Cirkeline - Hanne Hastrup',
 'Cirkeline på opdagelse - Ulla Raben',
 'Cirkeline godnat - Hanne Hastrup',
 'Den store bog om Cirkeline - Hanne Hastrup',
 'Cirkeline flytter til byen - Hanne Hastrup',
 'Kender du Pippi Langstrømpe? : billedbog - Ingrid Vang Nyman',
 'Cirkeline - Hanne Hastrup',
 'Mumitrolden i sneen - Sami Kaarla',
 "Mumi's første vinterdag - Jansson Tove ant",
 'Alle Bamses ting - Anna Gellert Abelin',
 'Mumitrolden farer vild - Sami Kaarla',
 'Peter Plys - nye eventyr i Hundredemeterskoven - Shepard Ernest H. dkfig',
 'Bamses allergoeste gave - Anna Gellert Abelin',
 "Mumi's måneskinseventyr - Jansson Tove ant",
 'Bamse danser bamlet - Katrine Hauch-Fausbøll',
 'Pippi Langstrømpe i Humlegården : billedbog - Ingrid Vang Nyman',
 'En dag med Barbapapa - Annette Tison']

In [17]:
recommendations(580)[0:20]

['Cykelmyggen Egon - Flemming Quist Møller',
 'Den store bog om den glade løve - Louise Fatio',
 'Bennys badekar - Flemming Quist Møller',
 'Pandekagekagen - Sven Nordqvist',
 'Stakkels Peddersen - Sven Nordqvist',
 'Gok-gok i køkkenhaven - Sven Nordqvist',
 'Findus flytter hjemmefra - Sven Nordqvist',
 'Og hanen gol - Sven Nordqvist',
 'Paddington - Michael Bond',
 'Rasmus får besøg - Jørgen Clevin',
 'Onkel Karfunkel - Ib Spang Olsen',
 'Rævejagten - Sven Nordqvist',
 'Da Findus var lille og forsvandt - Sven Nordqvist',
 'Ud i det blå med Peddersen - Sven Nordqvist',
 'Tante Andante - Ib Spang Olsen',
 'Cykelmyggen og dansemyggen - Flemming Quist Møller',
 'Hvordan elefanten fik sin lange snabel - Jan Mogensen (f. 1945)',
 'Da Emil hældte dej over sin far - Björn Berg',
 'Hattejagten - Sven Nordqvist',
 'Hokus Pokus og andre børnerim - Ib Spang Olsen']

In [18]:
recommendations(149)[0:20]

['Turen går til Berlin - Therkelsen Kirstine',
 'Politikens visuelle guide - Berlin - Søndervang Allan edt',
 'Top 10 Berlin - Jürgen Scheunemann',
 'Turen går til Amsterdam - Anette Jorsal',
 'Turen går til Prag - Hans Kragh-Jacobsen',
 'Turen går til Hamburg og Nordtyskland - Jytte Flamsholt Christensen',
 'Turen går til Californien & det vestlige USA - Preben Hansen (f. 1956)',
 'Turen går til London - Gunhild Riske',
 'Politikens Kort og godt om Berlin - Charmetant Jim',
 'Politikens visuelle guide - Prag - Vladimír Soukup',
 'Turen går til Barcelona og Costa Brava - Ole Loumann',
 'Top 10 London - Roger Williams',
 'Top 10 Prag - Theodore Schwinke',
 'Berlin - Wieland Giebel',
 'Top 10 Barcelona - Sorensen Annelise',
 'Turen går til Paris - Aske Munck',
 'Turen går til USA - Preben Hansen (f. 1956)',
 'Politikens visuelle guide - Barcelona & Catalonien - Allan Søndervang',
 'Turen går til Polen - Hannibal Lise aut',
 'Turen går til Rom - Alfredo Tesio']

In [19]:
recommendations(126)[0:20]

['Alt om håndarbejdes strikkemagasin - ',
 'Kreativ strik - ',
 "Ingelise's strikkemagasin - ",
 'DROPS : strikkdesign - Garnstudio',
 'Alt om håndarbejdes symagasin - ',
 'Maries ideer : håndarbejde, strik, sy, hækl, bolig, inspiration - ',
 'Burda style - ',
 'Burda - ',
 'Burda modemagasin : verdensberømt mode - ',
 "Ingelise's symagasin - ",
 'Hobby-nyt : dit kreative univers - ',
 'Ingelise : dit kreative magasin : alt om mode, sy, strik, hækling, idéer, bolig, trend - ',
 '1/2-times strik : hvad kan du strikke på en halv time eller mindre? - Carol Meldrum',
 'Strikketips - Sys Fredens',
 'Strikkedukker - Nerjordet Arne aut ill',
 'Aschehougs strikkebog - Claire Crompton',
 'Strømper & sokker - Sys Fredens',
 'Strik og krymp : filt i vaskemaskinen - Birte Ꜳrtun',
 'Ottobre design : woman European sizes 34-52 - ',
 'Strik - huer, vanter, strømper og pandebånd - Rachel Søgꜳrd']

In [20]:
recommendations(29)[0:20]

['1Q84 - Haruki Murakami',
 'Trækopfuglens krønike - Haruki Murakami',
 'Efter midnat - Haruki Murakami',
 'En vild fårejagt - Haruki Murakami',
 'Sputnik min elskede - Haruki Murakami',
 'Dans, dans, dans - Haruki Murakami',
 'Efter skælvet - Haruki Murakami',
 'Kafka på stranden - Haruki Murakami',
 'Min kamp : roman - Karl Ove Knausgård',
 'New York trilogien - Paul Auster',
 'Norwegian wood - Haruki Murakami',
 'Sønden for grænsen og vesten for solen - Haruki Murakami',
 '100 års ensomhed - Gabriel García Márquez',
 'Alt bliver oplyst : en roman - Jonathan Safran Foer',
 'Hvad jeg taler om når jeg taler om at løbe - Haruki Murakami',
 'Korrektioner - Jonathan Franzen',
 'Frihed - Jonathan Franzen',
 'Nattog til Lissabon : roman - Pascal Mercier',
 'Idioten - F. M. Dostojevskij',
 'Usynlig : roman - Paul Auster']

Vi har hermed lavet koden for en lille anbefalingsservice. For perspektivering, samlign eksempelvis resultaterne med anbefalingerne på bibliotekernes hjemmesider. Husk også at vi her kun kigger på et lille udvalg af materialebestanden. På bibliotek.dk kan anbefalinger findes ved, at fremsøge en bog, og derefter klikke på "Inspiration", og "Andre der har lånt...". Genrerummet som vi benytter her, er faktisk udregnet fra (en lille delmængde af) de "Andre der har lånt"-data som DBC havde med på Hack4DK.